In [1]:
from metrics import *
from bootstrap import *
from metric_classes import *

<function condition_number_xtx at 0x7fb0600d9048>


In [2]:
str(Cp())

"Mallow's Cp"

In [3]:
c = Condition_Number()
#c.evaluate(model)

In [49]:
class EvaluateInfo():
    def __init__(self, model, metrics = [], comparisons = [], characteristics = []):
        self.model = model
        self.characteristics = characteristics # characteristic of the fitted model itself
        self.metrics = metrics # compares y and y_pred
        self.comparisons = comparisons #compares y, y_pred, y_pred_with_reduced_features
        
    def _initResult(self):
        self.result = np.zeros((len(self.metrics) + len(self.comparisons) 
                                + len(self.characteristics), len(self.masks), self.n_samples))

    def _calcQuality(self):     
        self.quality = np.zeros((len(self.metrics) + len(self.comparisons) 
                                + len(self.characteristics), len(self.masks)))
        model = self.model
        for (m, mask) in enumerate(self.masks):
            reduced_X_test = self.X_test[:, mask]
            model.fit(self.X_train.T[mask].T, self.y_train)
            for (i, metric) in enumerate(self.metrics):
                self.quality[i][m] = metric.evaluate(model, reduced_X_test, self.y_test)

            for (i, comp) in enumerate(self.comparisons):
                ind = i + len(self.metrics)
                self.quality[ind][m] = comp.evaluate(self.full, model, self.X_test,
                                                                 reduced_X_test, self.y_test)
            for (i, char) in enumerate(self.characteristics):
                ind = i + len(self.metrics) + len(self.comparisons)
                self.quality[ind][m] = char.evaluate(model)
        
    def fit(self, X_train, y_train, X_test, y_test, masks = None, n_samples=20, len_sample=None):
        if masks is None:
            masks = np.ones((1, X.shape[1]), dtype=bool)
        masks = np.array(masks, dtype=bool)
        if len(masks.shape) == 1:
            masks = masks.reshape((1, len(masks.shape)))
        self.masks = masks
        
        if len_sample is None:
            len_sample = X_train.shape[1]
        self.len_sample = len_sample
        
        self.n_samples = n_samples

        self.X_train = X_train
        self.y_train = y_train

        self.X_test = X_test
        self.y_test = y_test
        
        self.full = deepcopy(self.model.fit(self.X_train, self.y_train))
        
        self._initResult()
        self._calcQuality()
    
    
    def get_quality(self):
        return self.quality
    
    def get_result(self):
        return self.result
    

In [64]:
from copy import deepcopy
class EvaluateStaticInfo(EvaluateInfo):
    def __init__(self, model, metrics = [], comparisons = []):
        super(EvaluateStaticInfo, self).__init__(model, metrics, comparisons, [])
    
    def __eval(self):
        model = self.model
        
        sample_X, sample_y = self.boot.values()
        
        self.models = []
        for mask in self.masks:
            self.models += [deepcopy(model.fit(self.X_train[:, mask], self.y_train))]
        
        for (m, mask) in enumerate(self.masks):
            for it in range(self.n_samples):
                reduced_X_cur = (sample_X[it].T[mask]).T
                for (i, metric) in enumerate(self.metrics):
                    self.result[i][m][it] = metric.evaluate(self.models[m], reduced_X_cur, sample_y[it])
                for(i, comp) in enumerate(self.comparisons):
                    ind = i + len(self.metrics)
                    self.result[ind][m][it] = comp.evaluate(self.full, self.models[m], sample_X[it], 
                                                              reduced_X_cur, sample_y[it])

    def fit(self, X_train, y_train, X_test, y_test, masks = None, n_samples=20, len_sample=None):
        super(EvaluateStaticInfo, self).fit(X_train, y_train, X_test, y_test, masks, n_samples, len_sample)
        self.boot = Bootstrap(self.X_test, self.y_test, self.n_samples, self.len_sample)
        self.__eval()
        

In [65]:
from copy import deepcopy
class EvaluateDynamicInfo(EvaluateInfo):
    def __init__(self, model, metrics = [], comparisons = [], characteristics = []):
        super(EvaluateDynamicInfo, self).__init__(model, metrics, comparisons, characteristics)
    
    def __eval(self):
        model = self.model
        
        sample_X, sample_y = self.boot.values()
            
        for (m, mask) in enumerate(self.masks):
            reduced_X_test = (self.X_test.T[mask]).T
            for it in range(self.n_samples):
                model.fit((sample_X[it].T[mask]).T, sample_y[it])
                for (i, metric) in enumerate(self.metrics):
                    self.result[i][m][it] = metric.evaluate(model, reduced_X_test, self.y_test)

                for (i, char) in enumerate(self.characteristics):
                    ind = i + len(self.metrics)
                    self.result[ind][m][it] = char.evaluate(model)

                for (i, comp) in enumerate(self.comparisons):
                    ind = i + len(self.metrics) + len(self.comparisons)
                    self.result[ind][m][it] = comp.evaluate(self.full, model, self.X_test,
                                                              reduced_X_test, self.y_test)
                

    def fit(self, X_train, y_train, X_test, y_test, masks = None, n_samples=20, len_sample=None):
        super(EvaluateDynamicInfo, self).fit(X_train, y_train, X_test, y_test, masks, n_samples, len_sample)
        self.boot = Bootstrap(self.X_test, self.y_test, self.n_samples, self.len_sample)
        self.__eval()
    

In [66]:
from sklearn.linear_model import LinearRegression
mo = LinearRegression()

In [67]:
import scipy.stats as sps

XX = sps.norm.rvs(size=(10, 5))
X2 = sps.norm.rvs(size=(10, 5))
zz = sps.uniform.rvs(size=(5, 3))
yy = XX @ zz + sps.norm.rvs(scale=0.1, size=(10, 3))
y2 = X2 @ zz + sps.norm.rvs(scale=0.1, size=(10, 3))
maskk = np.ones(XX.shape[1])
maskk[2]=0
maskk[1] = 0
maskk[3] = 0
print(maskk)

[ 1.  0.  0.  0.  1.]


In [68]:
metric1 = RSS()
metric2 = Adjusted_Determination_Coefficient()
metric3 = Determination_Coefficient()
metric4 = VIF()
metric5 = BIC()
char1 = Condition_Number()
comp1 = Cp()

In [69]:
info = EvaluateStaticInfo(mo, [metric1, metric2, metric3, metric4, metric5],
                           [comp1])#, [char1])

In [70]:
info.fit(XX, yy, X2, y2, masks=[np.ones(len(XX.T)), maskk])

In [71]:
info.get_result()

array([[[  6.12961434e-01,   7.03005187e-01,   3.10206129e-01,
           7.62816436e-01,   7.85673055e-01,   6.18924343e-01,
           8.65969786e-01,   7.93847452e-01,   4.25465497e-01,
           7.87355970e-01,   8.79604688e-01,   7.19452362e-01,
           6.94581983e-01,   7.12905902e-01,   5.30853756e-01,
           5.93897124e-01,   7.80339220e-01,   4.93764259e-01,
           7.71776409e-01,   5.80333914e-01],
        [  2.25144682e+01,   2.26574296e+01,   2.21665379e+01,
           2.14584411e+01,   4.15344403e+01,   2.81185210e+01,
           2.96608936e+01,   3.37535039e+01,   1.90793264e+01,
           3.27141414e+01,   3.73652266e+01,   1.75218252e+01,
           2.12388763e+01,   2.92376269e+01,   1.69759980e+01,
           2.36749468e+01,   2.48505990e+01,   2.46255352e+01,
           3.31363266e+01,   2.75272061e+01]],

       [[  9.62147803e-01,   9.88267822e-01,   9.88556165e-01,
           9.87567648e-01,   9.83788457e-01,   9.88406205e-01,
           9.43006896e-0

In [72]:
info.get_quality()

array([[  1.41733867e+00,   5.17683809e+01],
       [  9.73490673e-01,   3.17452162e-02],
       [  9.65916579e-01,  -2.44899008e-01],
       [  3.77225718e+01,   1.03278602e+00],
       [  1.29302641e+01,   5.63735511e+01],
       [  1.00000000e+00,   3.05250606e+01]])